In [1]:
import os
import torch
import random
import warnings
import math

import numpy as np
import pandas as pd
import torch.backends.cudnn as cudnn
import segmentation_models_pytorch as smp

from collections import Counter
from tqdm import tqdm
from torch.utils.data import DataLoader, sampler
from sklearn.model_selection import train_test_split
from utils import *
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
from efficientnet_pytorch.model import EfficientNet
import torchvision.models as models

warnings.filterwarnings("ignore")
seed = 99
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.set_default_tensor_type("torch.FloatTensor")
cudnn.benchmark = True

In [2]:
train_batch_size = 20
val_batch_size = 10
lr = 0.001

In [3]:
df = pd.read_csv('../train2.csv')
train_dir = '../stage2/train512/'

In [4]:
df.head()

,name,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any
0,ID_12cadc6af,0,0,0,0,0,0
1,ID_38fd7baa0,0,0,0,0,0,0
2,ID_6c5d82413,0,0,0,0,0,0
3,ID_aec8e68b3,0,0,0,1,0,1
4,ID_4d9209c7c,0,0,0,0,0,0


In [5]:
print('epidural:',df['epidural'].sum())
print('intraparenchymal:',df['intraparenchymal'].sum())
print('intraventricular:',df['intraventricular'].sum())
print('subarachnoid:',df['subarachnoid'].sum())
print('subdural:',df['subdural'].sum())
print('any:',df['any'].sum())

epidural: 3145
intraparenchymal: 36118
intraventricular: 26205
subarachnoid: 35675
subdural: 47166
any: 107933


In [6]:
train_df, val_df = train_test_split(df, test_size=0.1)

In [7]:
train_df.head()

,name,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any
529626,ID_a990ca7de,0,0,0,0,0,0
294818,ID_9aea2d035,0,0,0,0,0,0
675930,ID_b9fcc071b,0,0,0,0,0,0
199604,ID_fa7ab987f,0,0,0,0,0,0
463177,ID_cf9126b8d,0,0,0,0,0,0


In [8]:
train_set = RSNADataset(train_df,train_dir,make_transforms('train'))
val_set = RSNADataset(val_df,train_dir,make_transforms('val'))

In [9]:
# calculate class weight
if os.path.exists('weights_stage2.npy'):
    sample_weights = np.load('weights_stage2.npy').tolist()
    train_sampler = sampler.WeightedRandomSampler(sample_weights,len(sample_weights),replacement=True)
else:
    class_weight = 10000./np.array([2761, 32564, 23766, 32122, 42496, 97103])
    #class_weight = list(map(lambda x: x ** 0.6, class_weight))
    print(class_weight)
    sample_weights = []
    tqdm(df.iterrows(), total=df.shape[0])
    for idx, data in tqdm(train_df.iterrows(), total=train_df.shape[0]):
        sample_weight = 0
        sample_classes = 0
        for i in range(1, 7):
            if data[i] == 1:
                sample_classes += 1
                sample_weight += class_weight[i-1]
        if sample_classes == 0:
            sample_weight = 0.5
        sample_weights.append(sample_weight)

    train_sampler = sampler.WeightedRandomSampler(sample_weights,len(sample_weights),replacement=True)
    np.save('weights_stage2.npy', sample_weights)


In [10]:
train_loader = DataLoader(
    train_set,
    batch_size=train_batch_size,
    num_workers=4,
    pin_memory=True,
    sampler=train_sampler
)
val_loader = DataLoader(
    val_set,
    batch_size=val_batch_size,
    num_workers=4,
    pin_memory=True
)

In [11]:
# models
device = torch.device("cuda:0")
model = EfficientNet.from_pretrained('efficientnet-b2',num_classes=6).cuda()
model.eval()
print('Model build!')

Loaded pretrained weights for efficientnet-b2
Model build!


In [12]:
# optimizers
optimizer_1 = RAdam(model.parameters(), lr=0.001)
optimizer_2 = torch.optim.SGD(model.parameters(), lr=0.01)

In [13]:
# schedulers
scheduler = CosineAnnealingLR(optimizer_1, 3, eta_min=0.001, last_epoch=-1)

In [14]:
# hook functions
def after_epoch_end(payload, trainer, **kwargs):
    scheduler.step(payload['train_loss'])

In [17]:
from minetorch import Miner
from minetorch.metrics import *
from minetorch.loss import *

miner = Miner(
    code="fold-1",
    alchemistic_directory='./efficientnet-b2/',
    max_epochs=5,
    model=model,
    optimizer=optimizer_1,
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    loss_func=torch.nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([1.,1.,1.,1.,1.,2.]).cuda()),
    in_notebook=True,
    metrics=[precision, recall],
    hooks={
        'after_epoch_end': after_epoch_end,
    },
    resume=True,
    trival=False,
    gpu=True
)

miner.train()

RuntimeError: CUDA out of memory. Tried to allocate 976.00 MiB (GPU 1; 10.73 GiB total capacity; 7.92 GiB already allocated; 979.88 MiB free; 263.70 MiB cached)